In [114]:
import chardet


ModuleNotFoundError: No module named 'chardet'

In [113]:
pip install chardet

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement chardet (from versions: none)
ERROR: No matching distribution found for chardet
Note: you may need to restart the kernel to use updated packages.


In [105]:
import pandas as pd
import numpy as np
import re
# import matplotlib.pyplot as plt
# import seaborn as sns
# import mysql.connector
from sqlalchemy import create_engine

# import dateutil.parser
from dateutil.parser import parse
# from dateutil.parser import parse
# from fuzzywuzzy import process
import re
# from ast import literal_eval
import ast


In [106]:
"""Import and load the data into a dataframe."""
data = pd.read_csv("books_1.Best_Books_Ever.csv")
df = pd.DataFrame(data)

In [111]:
# Attempt to read the file using different encodings
encodings = ['utf-8', 'ISO-8859-1', 'windows-1252']
successful_encoding = None
error_messages = {}

for enc in encodings:
    try:
        temp_df = pd.read_csv("books_1.Best_Books_Ever.csv", encoding=enc)
        successful_encoding = enc
        break
    except Exception as e:
        error_messages[enc] = str(e)

successful_encoding, error_messages


('utf-8', {})

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

In [ ]:
# Custom function to split and extract the first part
def split_and_get_first(s):
    if '.' in s:
        return s.split('.')[0]
    elif '-' in s:
        return s.split('-')[0]
    else:
        return s

In [ ]:
def convert_string_to_list(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    return x

In [ ]:
def standardize_date_format(date_str):
    """
    Convert various date string formats into a standard format.
    """
    try:
        parsed_date = parse(date_str, fuzzy=True).date()
        
        # If the parsed year is in the future, adjust to the 20th century
        if parsed_date.year > 2018:
            parsed_date = parsed_date.replace(year=parsed_date.year - 100)
        
        return parsed_date
    except:
        # If parsing fails, return the original string
        return None

In [ ]:
def df_to_lowercase(df):
    """
    Convert all string values in the dataframe to lowercase.
    
    Parameters:
    - df: pandas DataFrame
    
    Returns:
    - DataFrame with all string values converted to lowercase
    """
    
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.lower()
    return df

In [ ]:
def analyze_categorical_column(dataframe, column_name):
    """Analyze a categorical column of a given DataFrame."""
    analysis_results = {}
    
    # Check if the column exists in the DataFrame
    if column_name not in dataframe.columns:
        return f"The column '{column_name}' does not exist in the DataFrame."
    
    # 1. Check for missing values
    analysis_results["missing_values"] = dataframe[column_name].isnull().sum()

    # 2. Check for uniqueness
    analysis_results["unique_values"] = dataframe[column_name].nunique()
    analysis_results["total_values"] = dataframe[column_name].count()
    analysis_results["duplicated_values"] = dataframe[column_name].duplicated().sum()

    # 3. Check for values with leading or trailing whitespace
    analysis_results["values_with_whitespace"] = dataframe[column_name].str.strip().ne(dataframe[column_name]).sum()

    # 4. Check for values with unusual characters (non-printable ASCII)
    analysis_results["values_with_unusual_chars"] = dataframe[column_name].apply(lambda x: any(ord(char) < 32 or ord(char) > 126 for char in str(x))).sum()

    # 5. Get the distribution of value lengths
    value_lengths = dataframe[column_name].str.len()
    analysis_results["shortest_value_length"] = value_lengths.min()
    analysis_results["longest_value_length"] = value_lengths.max()
    analysis_results["average_value_length"] = value_lengths.mean()
    
    shortest_value = dataframe[dataframe[column_name].str.len() == analysis_results['shortest_value_length']]
    longest_value = dataframe[dataframe[column_name].str.len() == analysis_results['longest_value_length']]
    
    return analysis_results, shortest_value, longest_value

In [ ]:
def extract_authors(author_str):
    """
    Extract main authors from the author string while considering additional non-author roles.
    """
    # Split by comma and strip whitespace
    names = [name.strip() for name in author_str.split(',')]
    
    # Filter out non-authors and remove (Goodreads Author) tag
    authors = [re.sub(r"\((Goodreads Author|Author)\)", "", name).strip() for name in names if not any(f"({role})" in name for role in non_authors)]
    
    return ", ".join(authors)

In [ ]:
# Function to remove brackets and their content from strings
def remove_brackets_content(s):
    return re.sub(r'\(.*?\)', '', s).strip()

In [ ]:
# Function to select the most common language from the list based on the ranked list of most common languages
def most_common_lang(lang_str):
    languages = [lang.strip() for lang in str(lang_str).split(';')]
    
    # Find the first common language that matches the book's list of languages
    # for each book's list of languages, we iterate over our ranked list of common languages and find the first match using the next function with a generator expression.
    matched_language = next((lang for lang in language_rank if lang in languages), languages[0])
    # matched_language = next((lang for lang in languages if lang in language_rank), languages[0])
    return matched_language

In [ ]:
def impute_missing_values(df):
    """
    Impute missing values in the dataframe.
    For columns of type int and float, impute with median.
    For columns of type object (strings), impute with mode.
    
    Parameters:
    - df: pandas DataFrame to be imputed
    
    Returns:
    - Imputed pandas DataFrame
    """
    
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            # Impute with median for int and float columns
            df[col].fillna(df[col].median(), inplace=True)
        elif df[col].dtype == 'object':
            # Impute with mode for object (string) columns
            # df[col].fillna(df[col].mode()[0], inplace=True)
            df[col].fillna('unknown', inplace=True)
    return df

In [ ]:
def winsorize(df, column, lower_quantile=0.05, upper_quantile=0.95):
    """Applying winsorization to remove outliers by converting them into less extreme values.
    Clip method limits the values in the 'price' to lower and upper limits. 
    Values below the 5th and above 95th percentiles are replaced by the 5th/95th percentile values accordingly. 
    This does not remove the values, rather caps them at certain points. """
    lower_limit = round(df[column].quantile(lower_quantile))
    upper_limit = round(df[column].quantile(upper_quantile))
    df[column] = df[column].clip(lower=lower_limit, upper=upper_limit)
    return df

In [ ]:
def outliers_numerical_cols(df):
    """Plot boxplots for all numerical columns in the dataframe to visualize outliers."""
    
    # Select columns with numerical data types (float64 and int64)
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    
    num_cols = 3  # We want 3 columns in our grid of plots
    num_rows = len(numerical_columns) // num_cols
    num_rows += len(numerical_columns) % num_cols  # Add an extra row if needed

    plt.figure(figsize=(15, 5 * num_rows))

    # Plot boxplots for each numerical column
    for i, col in enumerate(numerical_columns, start=1):
        plt.subplot(num_rows, num_cols, i)
        sns.boxplot(x=df[col].dropna())  # dropping NA values for plotting
        plt.title(col)

    plt.tight_layout()
    plt.show()

In [ ]:
def distribution_numerical_cols(df):
    
    # Select columns with numerical data types (float64 and int64)
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    
    num_cols = 3  # We want 3 columns in our grid of plots
    num_rows = len(numerical_columns) // num_cols
    num_rows += len(numerical_columns) % num_cols  # Add an extra row if needed

    plt.figure(figsize=(15, 5 * num_rows))

    # Plot boxplots for each numerical column
    for i, col in enumerate(numerical_columns, start=1):
        plt.subplot(num_rows, num_cols, i)
        sns.histplot(df[col], bins=30, kde=True)
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

In [ ]:
def distribution_numerical_cols(df):
    # Select columns with numerical data types (float64 and int64)
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    
    num_cols = 3  # We want 3 columns in our grid of plots
    num_rows = len(numerical_columns) // num_cols
    num_rows += len(numerical_columns) % num_cols  # Add an extra row if needed

    plt.figure(figsize=(15, 5 * num_rows))

    # Plot boxplots for each numerical column
    for i, col in enumerate(numerical_columns, start=1):
        try:
            plt.subplot(num_rows, num_cols, i)
            sns.histplot(df[col], bins=30, kde=False)
            plt.title(f'Distribution of {col}')
            plt.xlabel(col)
            plt.ylabel('Frequency')
        except:
            return None

    plt.tight_layout()
    plt.show()

In [ ]:
# Function to check if a column has outliers
def outlier_cols(column_data):
    Q1 = column_data.quantile(0.25)
    Q3 = column_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((column_data < lower_bound) | (column_data > upper_bound)).any()

In [112]:
import chardet

def detect_file_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

file_path = "/path/to/your/file.csv"
encoding = detect_file_encoding(file_path)
print(f"Detected encoding: {encoding}")

# Reading the File with Detected Encoding
df = pd.read_csv(file_path, encoding=encoding)


'utf-8'

In [3]:
df = df.drop_duplicates()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52428 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52428 non-null  object 
 1   title             52428 non-null  object 
 2   series            23445 non-null  object 
 3   author            52428 non-null  object 
 4   rating            52428 non-null  float64
 5   description       51092 non-null  object 
 6   language          48627 non-null  object 
 7   isbn              52428 non-null  object 
 8   genres            52428 non-null  object 
 9   characters        52428 non-null  object 
 10  bookFormat        50955 non-null  object 
 11  edition           4949 non-null   object 
 12  pages             50085 non-null  object 
 13  publisher         48736 non-null  object 
 14  publishDate       51549 non-null  object 
 15  firstPublishDate  31125 non-null  object 
 16  awards            52428 non-null  object 
 17

In [5]:
"""Check for Nulls, NAs, NANs, and fix them if any"""
print(df.isnull().sum().sum())

126859


In [6]:
print(df.isna().any().sum())

12


In [7]:
print(f'This dataset includes {df.shape[0]} books')

This dataset includes 52428 books


In [8]:
df.head(10)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1
5,19063.The_Book_Thief,The Book Thief,NaN,Markus Zusak (Goodreads Author),4.37,Librarian's note: An alternate cover edition c...,English,9780375831003,"['Historical Fiction', 'Fiction', 'Young Adult...","['Liesel Meminger', 'Hans Hubermann', 'Rudy St...",...,09/01/05,['National Jewish Book Award for Children’s an...,1834276,"['1048230', '524674', '186297', '48864', '26211']",96.0,"['Molching (Germany)', 'Germany']",https://i.gr-assets.com/images/S/compressed.ph...,1372809,14168,3.8
6,170448.Animal_Farm,Animal Farm,NaN,"George Orwell, Russell Baker (Preface), C.M. W...",3.95,Librarian's note: There is an Alternate Cover ...,English,9780451526342,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...","['Snowball', 'Napoleon', 'Clover', 'Boxer', 'O...",...,08/17/45,"['Prometheus Hall of Fame Award (2011)', 'Retr...",2740713,"['986764', '958699', '545475', '165093', '84682']",91.0,"['England', 'United Kingdom']",https://i.gr-assets.com/images/S/compressed.ph...,1276599,13264,4.42
7,11127.The_Chronicles_of_Narnia,The Chronicles of Narnia,The Chronicles of Narnia (Publication Order) #1–7,"C.S. Lewis, Pauline Baynes (Illustrator)",4.26,"Journeys to the end of the world, fantastic cr...",English,9999999999999,"['Fantasy', 'Classics', 'Fiction', 'Youn

In [9]:
unique_dates = df['firstPublishDate'].dropna().unique()
unique_dates

array(['06/21/03', '07/11/60', '01/28/13', ..., 'March 30th 2013',
       'August 3rd 2011', 'April 5th 2011'], dtype=object)

In [10]:
# Custom function to split and extract the first part
def split_and_get_first(s):
    if '.' in s:
        return s.split('.')[0]
    elif '-' in s:
        return s.split('-')[0]
    else:
        return s

In [11]:
def convert_string_to_list(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    return x

In [12]:
def standardize_date_format(date_str):
    """
    Convert various date string formats into a standard format.
    """
    try:
        parsed_date = parse(date_str, fuzzy=True).date()
        
        # If the parsed year is in the future, adjust to the 20th century
        if parsed_date.year > 2018:
            parsed_date = parsed_date.replace(year=parsed_date.year - 100)
        
        return parsed_date
    except:
        # If parsing fails, return the original string
        return None

In [13]:
def analyze_categorical_column(dataframe, column_name):
    """Analyze a categorical column of a given DataFrame."""
    analysis_results = {}
    
    # Check if the column exists in the DataFrame
    if column_name not in dataframe.columns:
        return f"The column '{column_name}' does not exist in the DataFrame."
    
    # 1. Check for missing values
    analysis_results["missing_values"] = dataframe[column_name].isnull().sum()

    # 2. Check for uniqueness
    analysis_results["unique_values"] = dataframe[column_name].nunique()
    analysis_results["total_values"] = dataframe[column_name].count()
    analysis_results["duplicated_values"] = dataframe[column_name].duplicated().sum()

    # 3. Check for values with leading or trailing whitespace
    analysis_results["values_with_whitespace"] = dataframe[column_name].str.strip().ne(dataframe[column_name]).sum()

    # 4. Check for values with unusual characters (non-printable ASCII)
    analysis_results["values_with_unusual_chars"] = dataframe[column_name].apply(lambda x: any(ord(char) < 32 or ord(char) > 126 for char in str(x))).sum()

    # 5. Get the distribution of value lengths
    value_lengths = dataframe[column_name].str.len()
    analysis_results["shortest_value_length"] = value_lengths.min()
    analysis_results["longest_value_length"] = value_lengths.max()
    analysis_results["average_value_length"] = value_lengths.mean()
    
    shortest_value = dataframe[dataframe[column_name].str.len() == analysis_results['shortest_value_length']]
    longest_value = dataframe[dataframe[column_name].str.len() == analysis_results['longest_value_length']]
    
    return analysis_results, shortest_value, longest_value

In [14]:
def extract_authors(author_str):
    """
    Extract main authors from the author string while considering additional non-author roles.
    """
    # Split by comma and strip whitespace
    names = [name.strip() for name in author_str.split(',')]
    
    # Filter out non-authors and remove (Goodreads Author) tag
    authors = [re.sub(r"\((Goodreads Author|Author)\)", "", name).strip() for name in names if not any(f"({role})" in name for role in non_authors)]
    
    return ", ".join(authors)

In [15]:
# Function to remove brackets and their content from strings
def remove_brackets_content(s):
    return re.sub(r'\(.*?\)', '', s).strip()

In [16]:
# Function to select the most common language from the list based on the ranked list of most common languages
def most_common_lang(lang_str):
    languages = [lang.strip() for lang in str(lang_str).split(';')]
    
    # Find the first common language that matches the book's list of languages
    # for each book's list of languages, we iterate over our ranked list of common languages and find the first match using the next function with a generator expression.
    matched_language = next((lang for lang in language_rank if lang in languages), languages[0])
    # matched_language = next((lang for lang in languages if lang in language_rank), languages[0])
    return matched_language

In [17]:
analysis_results_dict = {}  # Dictionary to store the analysis results for each column

for column in df.columns:
    try:
        results, shortest, longest = analyze_categorical_column(df, column)
        analysis_results_dict[column] = {
            "analysis_results": results,
            "shortest_value": shortest,
            "longest_value": longest
        }
    except Exception as e:
        print(f"Error occurred for column '{column}': {e}")

Error occurred for column 'rating': Can only use .str accessor with string values!
Error occurred for column 'numRatings': Can only use .str accessor with string values!
Error occurred for column 'likedPercent': Can only use .str accessor with string values!
Error occurred for column 'bbeScore': Can only use .str accessor with string values!
Error occurred for column 'bbeVotes': Can only use .str accessor with string values!


In [18]:
# Applying the function to the 'firstPublishDate' column
# df['firstPublishDate'] = df['firstPublishDate'].apply(standardize_date_format)
df['firstPublishDate'] = pd.to_datetime(df['firstPublishDate'].apply(standardize_date_format), errors='coerce')
df['publishDate'] = pd.to_datetime(df['publishDate'].apply(standardize_date_format), errors='coerce')
# df['standardized_firstPublishDate'] = df['firstPublishDate'].apply(standardize_date_format)

# Checking the results
# df['firstPublishDate'].head(10)
df[['firstPublishDate','publishDate']].head(10)
# df[['firstPublishDate', 'standardized_firstPublishDate']].head(10)

,firstPublishDate,publishDate
0,NaT,2008-09-14
1,2003-06-21,2004-09-28
2,1960-07-11,2006-05-23
3,2013-01-28,2000-10-10
4,2005-10-05,2006-09-06
5,2005-09-01,2006-03-14
6,1945-08-17,1996-04-28
7,1956-10-28,2002-09-16
8,1955-10-20,2012-09-25
9,1936-06-30,1999-04-01


In [19]:
# Sometimes first publish date is newer than standard publish date, this means data in columns is inaccurate. To fix this we merge the columns into one, keeping only the older date. 
df['oldestPublishDate'] = df[['firstPublishDate', 'publishDate']].min(axis=1)

# Drop the original columns
df = df.drop(['firstPublishDate', 'publishDate'], axis=1)

df['oldestPublishDate'].head()

0   2008-09-14
1   2003-06-21
2   1960-07-11
3   2000-10-10
4   2005-10-05
Name: oldestPublishDate, dtype: datetime64[ns]

In [20]:
# Apply the custom function to the 'bookId' column
df['bookId'] = df['bookId'].apply(split_and_get_first)

# Convert the 'bookId' column to integers
df['bookId'] = df['bookId'].astype(int)

In [21]:
print(df['bookId'].count() - df['bookId'].nunique())

4


In [22]:
# Check for non-unique bookIds
df[df['bookId'].duplicated(keep=False)].sort_values(by='bookId')

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,oldestPublishDate
37401,60614,Diamond Dogs,NaN,Alan Watt (Goodreads Author),3.46,Neil Garvin is a seventeen year old living in ...,English,9780446677844,"['Fiction', 'Mystery', 'Coming Of Age']",[],...,['ALA Alex Award (2001)'],320,"['54', '100', '115', '42', '9']",84.0,[],https://i.gr-assets.com/images/S/compressed.ph...,87,1,6.27,2000-09-01
37455,60614,Diamond Dogs,NaN,Alan Watt (Goodreads Author),3.46,Neil Garvin is a seventeen year old living in ...,English,9780446677844,"['Fiction', 'Mystery', 'Coming Of Age']",[],...,['ALA Alex Award (2001)'],320,"['54', '100', '115', '42', '9']",84.0,[],https://i.gr-assets.com/images/S/compressed.ph...,87,1,6.06,2000-09-01
37399,635270,The Planet Pirates,Planet Pirates #1-3,"Anne McCaffrey, Elizabeth Moon, Jody Lynn Nye ...",4.09,THE PLANET PIRATES ENSLAVED SASSINAK AND DESTR...,English,9780671319625,"['Science Fiction', 'Fantasy', 'Fiction', 'Spa...","['Sassinak', 'Lunzie']",...,[],1287,"['515', '444', '269', '49', '10']",95.0,[],https://i.gr-assets.com/images/S/compressed.ph...,87,2,7.72,1993-11-06
37453,635270,The Planet Pirates,Planet Pirates #1-3,"Anne McCaffrey, Elizabeth Moon, Jody Lynn Nye ...",4.09,THE PLANET PIRATES ENSLAVED SASSINAK AND DESTR...,English,9780671319625,"['Science Fiction', 'Fantasy', 'Fiction', 'Spa...","['Sassinak', 'Lunzie']",...,[],1287,"['515', '444', '269', '49', '10']",95.0,[],https://i.gr-assets.com/images/S/compressed.ph...,87,2,7.74,1993-11-06
37423,975953,Time of the Dragons,Shike #1,Robert Shea,4.27,"Shike is a novel about two characters: Jebu, a...",English,9780515048742,"['Historical Fiction', 'Fiction', 'Fantasy', '...",[],...,[],496,"['237', '179', '62', '14', '4']",96.0,[],https://i.gr-assets.com/images/S/compressed.ph...,87,1,5.45,1981-06-01
37477,975953,Time of the Dragons,Shike #1,Robert Shea,4.27,"Shike is a novel about two characters: Jebu, a...",English,9780515048742,"['Historical Fiction', 'Fiction', 'Fantasy', '...",[],...,[],496,"['237', '179', '62', '14', '4']",96.0,[],https://i.gr-assets.com/images/S/compressed.ph...,87,1,5.46,1981-06-01
37422,24903989,Widzę cię,Trilogia dei sensi #1,Irene Cao,3.22,"Gdyby dało się uchwycić przyjemność, Elena rob...",Polish,9788379991587,"['Romance', 'Erotica', 'Contemporary Romance',...","['Elena Kyler', 'Leonardo']",...,[],1292,"['216', '321', '405', '235', '115']",73.0,['Venice (Italy)'],https://i.gr-assets.com/images/S/compressed.ph...,87,1,7.28,2013-11-06
37476,24903989,Widzę cię,Trilogia dei sensi #1,Irene Cao,3.22,"Gdyby dało się uchwycić przyjemność, Elena rob...",Polish,9788379991587,"['Romance', 'Erotica', 'Contemporary Romance',...","['Elena Kyler', 'Leonardo']",...,[],1292,"['216', '321', '405', '235', '115']",73.0,['Venice (Italy)'],https://i.gr-assets.com/images/S/compressed.ph...,87,1,7.32,2013-11-06


In [23]:
# Dropping duplicates based on the 'bookId' column
# df.drop_duplicates(subset='bookId', inplace=True)
df = df.drop_duplicates(subset='bookId')

# Checking again for non-unique bookIds
df[df['bookId'].duplicated(keep=False)]

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,oldestPublishDate


In [24]:
analysis_results_dict['title']['analysis_results']

{'missing_values': 0,
 'unique_values': 49927,
 'total_values': 52428,
 'duplicated_values': 2501,
 'values_with_whitespace': 1,
 'values_with_unusual_chars': 3624,
 'shortest_value_length': 1,
 'longest_value_length': 251,
 'average_value_length': 24.62922484168765}

In [25]:
analysis_results_dict['title']['shortest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
5291,94034.Q,Q,NaN,"Luther Blissett, Shaun Whiteside (Translator),...",4.18,"In 1517, Martin Luther nails his ninety-five t...",English,9780156031967,"['Historical Fiction', 'Fiction', 'Historical'...",[],...,10/28/99,['Guardian First Book Award Nominee for Longli...,6623,"['3131', '2131', '905', '309', '147']",93.0,[],https://i.gr-assets.com/images/S/compressed.ph...,555,7,23.63
18158,7100173,Т,NaN,"Victor Pelevin, Виктор Пелевин",3.90,"«t» — новый роман писателя, в эпоху которого с...",Russian,9785699375158,"['Fiction', 'Russia', 'Modern', 'Contemporary'...","['Граф Т.', 'Ариэль', 'Кнопф']",...,10/30/09,['Большая книга for 3rd place (2010)'],1337,"['464', '439', '307', '91', '36']",91.0,[],https://i.gr-assets.com/images/S/compressed.ph...,100,1,14.93
19241,4944389-l,L,NaN,Kristy Nelwan,3.92,"Ava Torino, twentysomethinggirl, yang bekerja ...",Indonesian,9789790254176,"['Romance', 'Novels', 'Fiction', 'Chick Lit', ...",['Ava Torino'],...,10/29/08,[],710,"['229', '264', '160', '42', '15']",92.0,['Indonesia'],https://i.gr-assets.com/images/S/compressed.ph...,100,1,NaN
22869,18339155-f,F,NaN,Daniel Kehlmann,3.66,Es ist der Sommer vor der Wirtschaftskrise. Ma...,German,9783498035440,"['Fiction', 'German Literature', 'Contemporary...",[],...,10/30/13,['Independent Foreign Fiction Prize Nominee (2...,2448,"['474', '994', '708', '225', '47']",89.0,[],https://i.gr-assets.com/images/S/compressed.ph...,99,1,4.35
26071,13423043-z,Z,NaN,"Vassilis Vassilikos, Βασίλης Βασιλικός",4.03,Το μυθιστόρημα-ντοκουμέντο για τη δολοφονία το...,"Greek, Modern (1453-)",9789603644613,"['Fiction', 'Greece', 'Politics', 'Historical ...",[],...,11/30/66,[],1005,"['348', '413', '187', '42', '15']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,97,1,NaN
27066,27808997-o,O,NaN,Eka Kurniawan,4.10,Tentang seekor monyet yang ingin menikah denga...,Indonesian,9786020325590,"['Fiction', 'Indonesian Literature', 'Novels',...",[],...,NaN,['Kusala Sastra Khatulistiwa Nominee for Prosa...,1254,"['420', '595', '194', '39', '6']",96.0,[],https://i.gr-assets.com/images/S/compressed.ph...,97,1,NaN
29002,49386971,Φ,NaN,Γεώργιος Φραγκάκης (Goodreads Author),4.40,Ο Φίλιππος βρίσκεται σε κώμα ύστερα από μια απ...,"Greek, Modern (1453-)",9999999999999,[],[],...,NaN,[],15,"['10', '2', '2', '1', '0']",93.0,[],https://i.gr-assets.com/images/S/compressed.ph...,96,1,NaN
30187,7737810-z,Z,NaN,Michael Thomas Ford,3.63,The First Rule of Torching: Cleanse with fire....,English,9780060737580,"['Zombies', 'Horror', 'Young Adult', 'Science ...",[],...,NaN,[],697,"['188', '187', '218', '84', '20']",85.0,[],https://i.gr-assets.com/images/S/compressed.ph...,95,1,8.07
36355,6526307-7,7,NaN,Cem Akaş,3.89,"""Tanrı"" mıdır yazar? Yoksa, romana sızmış, bal...",Turkish,9999999999999,"['Turkish Literature', 'Turkish', 'Novels', 'F...",['Hakan'],...,11/06/92,[],269,"['93', '87', '62', '20', '7']",90.0,"['Turkey', 'Istanbul (Turkey)']",https://i.gr-assets.com/images/S/compressed.ph...,88,1,NaN
41408,4912955,٧,NaN,"Ghazi A. Algosaibi, غازي عبد الرحمن القصيبي",3.84,رواية ساخرة لغازي القصيبيالنيل والفرات:بسخريته...,Arabic,9999999999999,"['Novels', 'Fiction', 'Unfinished']",[],...,02/01/98,[],2017,"['648', '684', '463', '154', '68']",89.0,[],https://i.gr-assets.com/images/S/compressed.ph...,79,1,NaN


In [26]:
analysis_results_dict['title']['longest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
21883,12862036-novels-by-robin-hobb-including,"Novels By Robin Hobb, including: Assassin's Ap...",NaN,Hephaestus Books,4.42,This book consists entirely of reprints of Wik...,NaN,9781244687097,['Fantasy'],[],...,NaN,[],73,"['44', '20', '7', '0', '2']",97.0,[],https://i.gr-assets.com/images/S/compressed.ph...,99,1,NaN


In [27]:
non_unique_titles = df[df['title'].duplicated(keep=False)].sort_values(by='title')

non_unique_titles

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,oldestPublishDate
385,11590,'Salem's Lot,NaN,Stephen King (Goodreads Author),4.03,Thousands of miles away from the small townshi...,English,9780450031069,"['Horror', 'Fiction', 'Vampires', 'Fantasy', '...","['Benjamin Mears', 'Susan Norton', 'Matt Burke...",...,['World Fantasy Award Nominee for Best Novel (...,334051,"['122020', '121352', '72184', '14697', '3798']",94.0,"[""Jerusalem's Lot, Maine (United States)""]",https://i.gr-assets.com/images/S/compressed.ph...,28986,393,1.12,1975-10-17
3674,5413,'Salem's Lot,NaN,"Stephen King (Goodreads Author), Jerry N. Uels...",4.25,"Stephen King's second novel, the vampire bests...",English,9780385516488,"['Horror', 'Fiction', 'Vampires', 'Fantasy', '...",[],...,[],94874,"['45186', '31994', '14804', '2313', '577']",97.0,[],https://i.gr-assets.com/images/S/compressed.ph...,873,11,23.39,1975-10-17
48062,16000978,'Til Death,Rebel Ridge #3,Sharon Sala (Goodreads Author),4.23,He left in handcuffs. Now it's time to set him...,English,9780778314271,"['Romantic Suspense', 'Romance', 'Suspense', '...","['Lincoln Fox', 'Meg Walker']",...,[],1008,"['476', '341', '149', '28', '14']",96.0,['Kentucky (United States)'],https://i.gr-assets.com/images/S/compressed.ph...,52,1,6.55,2013-01-01
27557,7171787,'Til Death,Secret Society #3,Miasha (Goodreads Author),4.20,Celess is back and hotter than ever in Miasha'...,English,9781416589884,['Fiction'],[],...,[],183,"['97', '44', '27', '12', '3']",92.0,[],https://i.gr-assets.com/images/S/compressed.ph...,96,1,6.08,2010-08-11
10578,21456940,1919,NaN,أحمد مراد (Goodreads Author),3.65,سيختطفك أحمد مراد في آلة زمن، ليهبط بك في حقبة...,Arabic,9789770932926,"['Novels', 'Fiction', 'Historical Fiction', 'H...","['سعد زغلول', 'أحمد كيرة', 'صفية زغلول', 'المل...",...,[],20860,"['5339', '6906', '5593', '2034', '988']",86.0,"['Egypt', 'Istanbul (Turkey)']",https://i.gr-assets.com/images/S/compressed.ph...,232,3,45.76,2014-04-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47342,16300950,Ποιήματα,NaN,"Kiki Dimoula, Κική Δημουλά",4.40,"Συγκεντρωτική έκδοση των ποιητικών συλλογών: ""...","Greek, Modern (1453-)",9789607721365,['Poetry'],[],...,[],192,"['102', '70', '15', '4', '1']",97.0,[],https://i.gr-assets.com/images/S/compressed.ph...,56,1,NaN,1998-10-06
27662,9736277,الأعمال الشعرية الكاملة,NaN,فؤاد قاعود,4.56,NaN,Arabic,9999999999999,[],[],...,[],9,"['6', '2', '1', '0', '0']",100.0,[],NaN,96,1,NaN,NaT
15465,6718801,الأعمال الشعرية الكاملة,NaN,"Mahmoud Darwish, محمود درويش",4.46,عندما تهاجر أسراب السنونو مواطنها... تسمع في ح...,Arabic,9999999999999,"['Poetry', 'Nonfiction', 'Literature']",[],...,[],2514,"['1623', '577', '211', '59', '44']",96.0,[],https://i.gr-assets.com/images/S/compressed.ph...,160,2,NaN,1973-08-30
11863,1618417,نهج البلاغه,NaN,"علي بن أبي طالب, الشريف الرضي (گردآوری), سید ج...",4.56,توضيح ناشرشك نيست كه ترجمه فنى است ظريف و دقيق...,Persian,9999999999999,"['Religion', 'Islam', 'Nonfiction', 'Philosoph...",[],...,['کتاب سال جمهوری اسلامی ایران (1991)'],1796,"['1372', '225', '94', '43', '62']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,196,2,NaN,1991-08-30


In [28]:
df['title'] = df['title'].str[:50]

In [29]:
analysis_results_dict['series']['analysis_results']

{'missing_values': 28983,
 'unique_values': 22802,
 'total_values': 23445,
 'duplicated_values': 29625,
 'values_with_whitespace': 29041,
 'values_with_unusual_chars': 868,
 'shortest_value_length': 1.0,
 'longest_value_length': 93.0,
 'average_value_length': 18.99104286628279}

In [30]:
analysis_results_dict['title']['shortest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
5291,94034.Q,Q,NaN,"Luther Blissett, Shaun Whiteside (Translator),...",4.18,"In 1517, Martin Luther nails his ninety-five t...",English,9780156031967,"['Historical Fiction', 'Fiction', 'Historical'...",[],...,10/28/99,['Guardian First Book Award Nominee for Longli...,6623,"['3131', '2131', '905', '309', '147']",93.0,[],https://i.gr-assets.com/images/S/compressed.ph...,555,7,23.63
18158,7100173,Т,NaN,"Victor Pelevin, Виктор Пелевин",3.90,"«t» — новый роман писателя, в эпоху которого с...",Russian,9785699375158,"['Fiction', 'Russia', 'Modern', 'Contemporary'...","['Граф Т.', 'Ариэль', 'Кнопф']",...,10/30/09,['Большая книга for 3rd place (2010)'],1337,"['464', '439', '307', '91', '36']",91.0,[],https://i.gr-assets.com/images/S/compressed.ph...,100,1,14.93
19241,4944389-l,L,NaN,Kristy Nelwan,3.92,"Ava Torino, twentysomethinggirl, yang bekerja ...",Indonesian,9789790254176,"['Romance', 'Novels', 'Fiction', 'Chick Lit', ...",['Ava Torino'],...,10/29/08,[],710,"['229', '264', '160', '42', '15']",92.0,['Indonesia'],https://i.gr-assets.com/images/S/compressed.ph...,100,1,NaN
22869,18339155-f,F,NaN,Daniel Kehlmann,3.66,Es ist der Sommer vor der Wirtschaftskrise. Ma...,German,9783498035440,"['Fiction', 'German Literature', 'Contemporary...",[],...,10/30/13,['Independent Foreign Fiction Prize Nominee (2...,2448,"['474', '994', '708', '225', '47']",89.0,[],https://i.gr-assets.com/images/S/compressed.ph...,99,1,4.35
26071,13423043-z,Z,NaN,"Vassilis Vassilikos, Βασίλης Βασιλικός",4.03,Το μυθιστόρημα-ντοκουμέντο για τη δολοφονία το...,"Greek, Modern (1453-)",9789603644613,"['Fiction', 'Greece', 'Politics', 'Historical ...",[],...,11/30/66,[],1005,"['348', '413', '187', '42', '15']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,97,1,NaN
27066,27808997-o,O,NaN,Eka Kurniawan,4.10,Tentang seekor monyet yang ingin menikah denga...,Indonesian,9786020325590,"['Fiction', 'Indonesian Literature', 'Novels',...",[],...,NaN,['Kusala Sastra Khatulistiwa Nominee for Prosa...,1254,"['420', '595', '194', '39', '6']",96.0,[],https://i.gr-assets.com/images/S/compressed.ph...,97,1,NaN
29002,49386971,Φ,NaN,Γεώργιος Φραγκάκης (Goodreads Author),4.40,Ο Φίλιππος βρίσκεται σε κώμα ύστερα από μια απ...,"Greek, Modern (1453-)",9999999999999,[],[],...,NaN,[],15,"['10', '2', '2', '1', '0']",93.0,[],https://i.gr-assets.com/images/S/compressed.ph...,96,1,NaN
30187,7737810-z,Z,NaN,Michael Thomas Ford,3.63,The First Rule of Torching: Cleanse with fire....,English,9780060737580,"['Zombies', 'Horror', 'Young Adult', 'Science ...",[],...,NaN,[],697,"['188', '187', '218', '84', '20']",85.0,[],https://i.gr-assets.com/images/S/compressed.ph...,95,1,8.07
36355,6526307-7,7,NaN,Cem Akaş,3.89,"""Tanrı"" mıdır yazar? Yoksa, romana sızmış, bal...",Turkish,9999999999999,"['Turkish Literature', 'Turkish', 'Novels', 'F...",['Hakan'],...,11/06/92,[],269,"['93', '87', '62', '20', '7']",90.0,"['Turkey', 'Istanbul (Turkey)']",https://i.gr-assets.com/images/S/compressed.ph...,88,1,NaN
41408,4912955,٧,NaN,"Ghazi A. Algosaibi, غازي عبد الرحمن القصيبي",3.84,رواية ساخرة لغازي القصيبيالنيل والفرات:بسخريته...,Arabic,9999999999999,"['Novels', 'Fiction', 'Unfinished']",[],...,02/01/98,[],2017,"['648', '684', '463', '154', '68']",89.0,[],https://i.gr-assets.com/images/S/compressed.ph...,79,1,NaN


In [31]:
analysis_results_dict['title']['longest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
21883,12862036-novels-by-robin-hobb-including,"Novels By Robin Hobb, including: Assassin's Ap...",NaN,Hephaestus Books,4.42,This book consists entirely of reprints of Wik...,NaN,9781244687097,['Fantasy'],[],...,NaN,[],73,"['44', '20', '7', '0', '2']",97.0,[],https://i.gr-assets.com/images/S/compressed.ph...,99,1,NaN


In [32]:
"""Most make no sense, thus proceeding to keep oply the series title."""
df['series'] = df['series'].str.split('#', expand = True)[0]
df['series'] = df['series'].str.strip()
df['series'].head

<bound method NDFrame.head of 0             The Hunger Games
1                 Harry Potter
2        To Kill a Mockingbird
3                          NaN
4            The Twilight Saga
                 ...          
52473                  Fateful
52474           Sense of Truth
52475           Soul Guardians
52476                      NaN
52477        Stripling Warrior
Name: series, Length: 52424, dtype: object>

In [33]:
df['series'] = df['series'].str[:30]

In [34]:
new_col = 'author'

analysis_results_dict[new_col]['analysis_results']

{'missing_values': 0,
 'unique_values': 28227,
 'total_values': 52428,
 'duplicated_values': 24201,
 'values_with_whitespace': 0,
 'values_with_unusual_chars': 4225,
 'shortest_value_length': 2,
 'longest_value_length': 381,
 'average_value_length': 30.109216449225606}

In [35]:
analysis_results_dict[new_col]['shortest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
14844,588066.Vice,Vice: New and Selected Poems,NaN,Ai,4.12,Collected here are poems from Ai's previous fi...,English,9780393320183,"['Poetry', 'Read For School', 'Womens', 'Liter...",[],...,03/01/99,['National Book Award for Poetry (1999)'],480,"['208', '154', '92', '21', '5']",95.0,[],https://i.gr-assets.com/images/S/compressed.ph...,169,2,5.22
30471,39938003,天觀雙俠（全四卷）,NaN,鄭丰,5.00,趙觀，從小在青樓成長的他，面貌清秀俊雅，個性卻圓滑狡黠、足智多謀，生父不詳的他在親娘被仇...,Chinese,9999999999999,[],[],...,NaN,[],1,[],NaN,[],https://i.gr-assets.com/images/S/compressed.ph...,95,1,NaN
36880,32156380-passion-heaven,Passion Heaven,NaN,籽月,3.29,Shu Ya-wang is a 28 years-old landscape archit...,Chinese,9999999999999,[],[],...,11/06/10,[],7,"['2', '1', '2', '1', '1']",71.0,[],https://i.gr-assets.com/images/S/compressed.ph...,88,1,NaN


In [36]:
analysis_results_dict[new_col]['longest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
47020,15980677-kicking-it,Kicking It,Alex Craft #2.5 - Ruby Red,"Faith Hunter (Goodreads Author) (Editor), Kala...",3.83,Featuring stories by FAITH HUNTER * KALAYNA PR...,English,9780451419002,"['Vampires', 'Paranormal', 'Fantasy', 'Antholo...","['Luc (Chicagoland Vampires)', 'Lindsey (Chica...",...,NaN,[],3064,"['823', '1110', '955', '152', '24']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,58,1,5.85


In [37]:
# Author roles
unique_authors = df['author'].dropna().unique()

# Extract roles using a regex pattern
roles_pattern = re.compile(r'\((.*?)\)')
all_roles = []

# For each unique author string, extract roles and append to the all_roles list
for author_str in unique_authors:
    roles_found = roles_pattern.findall(author_str)
    all_roles.extend(roles_found)

# Compile unique roles
unique_roles = list(set(all_roles))

# Define roles that are typically considered as authors
authors = ["Author", "Goodreads Author"]

# Create the extended_non_author_roles_saved list by removing the author roles
non_authors = [role for role in unique_roles if role not in authors]

# Display the first few roles from the saved list
non_authors


['Editor/Contributor',
 'تصدير',
 'Inker',
 'series creator',
 'Μεταφράστρια',
 'coord.',
 'prefazione',
 'Editor, Introduction',
 'Creator',
 'narrator',
 'Illustrated by',
 'Forward',
 'Producer',
 'Editor/Contributer',
 'K.S.R. Kingworth',
 'Introduction/Editor',
 'script, adaptation',
 'traducción',
 'Preface & Glossary',
 'وەرگێڕ',
 'Original Author',
 'Editor-in-charge',
 'Based on the TV series',
 'Senior Sub Editor',
 'Narrator ',
 'Annotations and Afterword',
 'Created by',
 'Book Design',
 'Assistant',
 'إعداد وتقديم',
 'Introduction and Notes',
 'Color',
 'Art by',
 'With',
 'introduction and screenplay',
 'Calligraphy',
 'Dreamer',
 'pseudonym',
 'Translator, contributor',
 'تشعیر',
 'تدقيق اللغة',
 'Presentación',
 'Source Material',
 'Colors',
 'Ilustração',
 'Critical Study',
 'Concept by',
 'Translator',
 'Afterword',
 'Abridged by',
 'Writer, Artist',
 'pseudonym ',
 'Translator, Parts 3 and 4',
 'Kata Pengantar',
 'Writer & Illustrator',
 'proofreader',
 'عبد الله محم

In [38]:
# Apply the function with additional non-author roles to the 'author' column
df['author'] = df['author'].dropna().apply(extract_authors)

# Display the results
df['author'].head(30)

0                                   Suzanne Collins
1                                      J.K. Rowling
2                                        Harper Lee
3                                       Jane Austen
4                                   Stephenie Meyer
5                                      Markus Zusak
6                                     George Orwell
7                                        C.S. Lewis
8                                    J.R.R. Tolkien
9                                 Margaret Mitchell
10                                       John Green
11                                    Douglas Adams
12                                 Shel Silverstein
13                                     Emily Brontë
14                                        Dan Brown
15                                    Arthur Golden
16                                      Oscar Wilde
17                                    Lewis Carroll
18                                 Charlotte Brontë
19          

In [39]:
# Filter the rows where 'mainAuthorsExtended' has more than one author
multi_authors_df = df[df['author'].str.count(',') == 0]

# Display the first 10 rows of the filtered dataframe
multi_authors_df['author'].head(10)

0      Suzanne Collins
1         J.K. Rowling
2           Harper Lee
3          Jane Austen
4      Stephenie Meyer
5         Markus Zusak
6        George Orwell
7           C.S. Lewis
8       J.R.R. Tolkien
9    Margaret Mitchell
Name: author, dtype: object

In [40]:
"""# Filter out empty strings from sorted_names
sorted_names = [name for name in sorted_names if name]

# Group potential matches by the first word in the sorted name (this reduces the number of comparisons)
first_word_to_names = {}
for name in sorted_names:
    first_word = name.split()[0]
    first_word_to_names.setdefault(first_word, []).append(name)

# Dictionary to store potential matches
potential_matches_reduced = {}

# Compare names within each group to find potential matches
for first_word, names in first_word_to_names.items():
    for i, name in enumerate(names):
        for j in range(i+1, len(names)):
            other_name = names[j]
            if fuzz.ratio(name, other_name) > 90:
                original_name = sorted_to_original[name]
                original_other_name = sorted_to_original[other_name]
                potential_matches_reduced.setdefault(original_name, []).append(original_other_name)

# Display the first few potential matches
dict(list(potential_matches_reduced.items())[:10])
"""

'# Filter out empty strings from sorted_names\nsorted_names = [name for name in sorted_names if name]\n\n# Group potential matches by the first word in the sorted name (this reduces the number of comparisons)\nfirst_word_to_names = {}\nfor name in sorted_names:\n    first_word = name.split()[0]\n    first_word_to_names.setdefault(first_word, []).append(name)\n\n# Dictionary to store potential matches\npotential_matches_reduced = {}\n\n# Compare names within each group to find potential matches\nfor first_word, names in first_word_to_names.items():\n    for i, name in enumerate(names):\n        for j in range(i+1, len(names)):\n            other_name = names[j]\n            if fuzz.ratio(name, other_name) > 90:\n                original_name = sorted_to_original[name]\n                original_other_name = sorted_to_original[other_name]\n                potential_matches_reduced.setdefault(original_name, []).append(original_other_name)\n\n# Display the first few potential matches\ndict(

In [41]:
# Stick to 2 authors
df['author'][df['author'].str.count(',') == 0].count(), df['author'][df['author'].str.count(',') == 1].count(), df['author'][df['author'].str.count(',') == 2].count()

(48338, 3467, 377)

In [ ]:
# Split the 'author' column by comma, select the first two authors, and join them back
df['author'] = df['author'].str.split(',').apply(lambda x: ', '.join(x[:2]) if isinstance(x, list) else x)

In [100]:
new_col = 'description'

analysis_results_dict[new_col]['analysis_results']

{'missing_values': 1336,
 'unique_values': 50888,
 'total_values': 51092,
 'duplicated_values': 1539,
 'values_with_whitespace': 3684,
 'values_with_unusual_chars': 25038,
 'shortest_value_length': 3.0,
 'longest_value_length': 24733.0,
 'average_value_length': 861.3337900258357}

In [43]:
analysis_results_dict[new_col]['shortest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
28236,5948950,شيء سيبقى بيننا,NaN,فاروق جويدة,3.90,شعر,Arabic,9999999999999,"['Poetry', 'Literature', 'Plays', 'Love']",[],...,10/30/83,[],2301,"['784', '758', '559', '152', '48']",91.0,[],https://i.gr-assets.com/images/S/compressed.ph...,96,1,NaN
28866,25657509-train-your-brain,Train Your Brain,NaN,Ryuta Kawashima,3.84,New,English,9788183224123,"['Nonfiction', 'Psychology', 'Science', 'Self ...",[],...,10/30/03,[],135,"['50', '38', '30', '10', '7']",87.0,[],https://i.gr-assets.com/images/S/compressed.ph...,96,1,3.47
43057,17231104,أحلام الفارس القديم,NaN,صلاح عبد الصبور,3.65,شعر,NaN,9999999999999,['Poetry'],[],...,11/05/64,[],325,"['73', '113', '99', '33', '7']",88.0,[],https://i.gr-assets.com/images/S/compressed.ph...,75,1,NaN
46255,5868421,الرسم بالكلمات,NaN,نزار قباني,3.94,شعر,Arabic,9999999999999,"['Poetry', 'Literature', 'Nonfiction', 'Love',...",[],...,11/06/66,[],2839,"['1039', '920', '625', '180', '75']",91.0,[],https://i.gr-assets.com/images/S/compressed.ph...,62,1,NaN


In [44]:
analysis_results_dict[new_col]['longest_value']

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
18912,17857619-spiritus-mundi,Spiritus Mundi,Spiritus Mundi #1,Robert Sheppard (Goodreads Author),4.00,"Spiritus Mundi by Robert Sheppard, nominated f...",English,9999999999999,[],[],...,NaN,[],14,"['7', '4', '1', '0', '2']",86.0,[],https://i.gr-assets.com/images/S/compressed.ph...,100,1,NaN
22017,17857704-spiritus-mundi---the-romance,Spiritus Mundi - The Romance,Spiritus Mundi #2,Robert Sheppard (Goodreads Author),3.92,"Spiritus Mundi by Robert Sheppard, nominated f...",English,B00CGM8BZG,[],[],...,NaN,[],13,"['6', '4', '1', '0', '2']",85.0,[],https://i.gr-assets.com/images/S/compressed.ph...,99,1,NaN


In [101]:
# Convert all values in 'description' column to lowercase
df['description'] = df['description'].str.lower()

# Trim whitespace from the 'description' column
df['description'] = df['description'].str.strip()

# Set maximum length for the 'description' column to 1000 characters and truncate the rest
df['description'] = df['description'].str[:1000]

In [102]:
df['description'].str.len().max()

1000.0

In [46]:
df[['bookId', 'title', 'description']].head(10)

,bookId,title,description
0,2767052,The Hunger Games,winning means fame and fortune.losing means ce...
1,2,Harry Potter and the Order of the Phoenix,there is a door at the end of a silent corrido...
2,2657,To Kill a Mockingbird,the unforgettable novel of a childhood in a sl...
3,1885,Pride and Prejudice,alternate cover edition of isbn 9780679783268s...
4,41865,Twilight,about three things i was absolutely positive.\...
5,19063,The Book Thief,librarian's note: an alternate cover edition c...
6,170448,Animal Farm,librarian's note: there is an alternate cover ...
7,11127,The Chronicles of Narnia,"journeys to the end of the world, fantastic cr..."
8,30,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,"this four-volume, boxed set contains j.r.r. to..."
9,18405,Gone with the Wind,"scarlett o'hara, the beautiful, spoiled daught..."


In [47]:
new_col = 'language'

analysis_results_dict[new_col]['analysis_results']

{'missing_values': 3801,
 'unique_values': 81,
 'total_values': 48627,
 'duplicated_values': 52346,
 'values_with_whitespace': 3801,
 'values_with_unusual_chars': 12,
 'shortest_value_length': 4.0,
 'longest_value_length': 37.0,
 'average_value_length': 7.053118637793818}

In [48]:
analysis_results_dict[new_col]['shortest_value'][new_col]

4636     Urdu
6036     Urdu
9940     Urdu
13660    Urdu
13992    Urdu
         ... 
43007    Urdu
43318    Urdu
43603    Urdu
46040    Urdu
46327    Urdu
Name: language, Length: 63, dtype: object

In [49]:
analysis_results_dict[new_col]['longest_value'][new_col]

22126    Aromanian; Arumanian; Macedo-Romanian
51230    Norwegian Nynorsk; Nynorsk, Norwegian
Name: language, dtype: object

In [50]:
df['language'] = df['language'].str.strip()

In [51]:
# Extract all languages from the 'language' column
all_languages = df['language'].str.split(';', expand=True).stack().str.strip()

language_rank = all_languages.value_counts().index.tolist()
language_rank

['English',
 'Arabic',
 'Spanish',
 'French',
 'German',
 'Indonesian',
 'Portuguese',
 'Italian',
 'Dutch',
 'Turkish',
 'Persian',
 'Polish',
 'Greek, Modern (1453-)',
 'Russian',
 'Swedish',
 'Japanese',
 'Romanian',
 'Bengali',
 'Malay',
 'Bulgarian',
 'Danish',
 'Urdu',
 'Finnish',
 'Czech',
 'Pilipino',
 'Filipino',
 'Lithuanian',
 'Serbian',
 'Norwegian',
 'Tamil',
 'Croatian',
 'Estonian',
 'Georgian',
 'Vietnamese',
 'Marathi',
 'Hindi',
 'Hungarian',
 'Multiple languages',
 'Bokmål, Norwegian',
 'Norwegian Bokmål',
 'Catalan',
 'Valencian',
 'Malayalam',
 'Chinese',
 'Nepali',
 'Slovak',
 'Latvian',
 'Tagalog',
 'Korean',
 'Farsi',
 'Albanian',
 'Armenian',
 'Gujarati',
 'Bosnian',
 'Kannada',
 'Ukrainian',
 'Thai',
 'Icelandic',
 'Amharic',
 'Slovenian',
 'Panjabi',
 'Afrikaans',
 'Macedonian',
 'Maltese',
 'Greek, Ancient (to 1453)',
 'Telugu',
 'Punjabi',
 'Galician',
 'Undetermined',
 'Aleut',
 'Mongolian',
 'Mayan languages',
 'Duala',
 'Australian languages',
 'Norwegia

In [52]:
# Apply the function to the 'language' column
df['language'] = df['language'].apply(most_common_lang)
df['language'] = df['language'].str.replace(f',', '', regex=True)

# Display the first few rows to verify the changes
df[['bookId', 'language']].tail(20)#.iloc[22126]


,bookId,language
52458,2147003,English
52459,797498,English
52460,2037584,English
52461,22925176,English
52462,2613071,Persian
52463,25876358,English
52464,36374396,nan
52465,6403092,English
52466,8756401,English
52467,40510415,English


In [53]:
# language_ranks = {lang: language_rank.index(lang) + 1 for lang in ["Aromanian", "Arumanian", "Macedo-Romanian"] if lang in language_rank}
language_ranks = {lang: language_rank.index(lang) + 1 for lang in ["Aromanian", "Arumanian", "Macedo-Romanian"] if lang in language_rank}
language_ranks


{'Aromanian': 83, 'Arumanian': 82, 'Macedo-Romanian': 81}

In [54]:
new_col = 'bookFormat'

analysis_results_dict[new_col]['analysis_results']

{'missing_values': 1473,
 'unique_values': 136,
 'total_values': 50955,
 'duplicated_values': 52291,
 'values_with_whitespace': 1478,
 'values_with_unusual_chars': 17,
 'shortest_value_length': 2.0,
 'longest_value_length': 33.0,
 'average_value_length': 9.995917966833481}

In [55]:
analysis_results_dict[new_col]['shortest_value'][new_col]

26921    文庫
36488    文庫
Name: bookFormat, dtype: object

In [56]:
analysis_results_dict[new_col]['longest_value'][new_col]

14071    Science Fiction Book Club Omnibus
Name: bookFormat, dtype: object

In [57]:
df['bookFormat'] = df['bookFormat'].str.strip().str.lower()

df['bookFormat'] = df['bookFormat'].fillna('unknown')

# Display the unique values for review after cleaning
df['bookFormat'].value_counts()


bookFormat
paperback                26779
hardcover                12266
kindle edition            5824
mass market paperback     2567
ebook                     2545
                         ...  
box-set                      1
paperback, ebook             1
paper book                   1
bonded leather               1
gebunden                     1
Name: count, Length: 120, dtype: int64

In [58]:
# Handling Multiple Formats
# For this implementation, we'll keep the first mentioned format for simplicity
df['bookFormat'] = df['bookFormat'].str.split(',').str[0]

In [59]:
# Create mapping list for cases where count exceeds 100. Could be extended to include all cases or uze approximate matching
format_mapping = {
    'e-book': 'ebook',
    'electronic book': 'ebook',
    'ebook': 'ebook',
    'pbk.': 'paperback',
    'soft cover': 'paperback',
    'paperback': 'paperback',
    'mass market paperback': 'paperback',
    'trade paper': 'paperback',
    'book': 'hardcover',
    'hardcover': 'hardcover',
    'hard cover': 'hardcover',
    'board book': 'hardcover',
    'kindle edition': 'kindle edition',
    'audiobook': 'audiobook'
}
"""The .get() method for dictionaries returns the value for a given key. If the key doesn't exist, it returns a default value. 
If a particular value doesn't have a mapping in the dictionary, it's converted to 'other'."""
df['bookFormat'] = df['bookFormat'].apply(lambda x: format_mapping.get(x, 'other'))
# df['bookFormat'] = df['bookFormat'].apply(lambda x: format_mapping[x] if x in format_mapping else 'other')
# df['bookFormat'] = df['bookFormat'].replace(format_mapping, inplace=True)
# df['bookFormat'] = df['bookFormat'].replace(format_mapping)

# Display the unique values for review after the refined cleaning
df['bookFormat'].value_counts()


bookFormat
paperback         29349
hardcover         12307
kindle edition     5824
ebook              2545
other              2292
audiobook           107
Name: count, dtype: int64

In [60]:
new_col = 'edition'

analysis_results_dict[new_col]['analysis_results']

{'missing_values': 47479,
 'unique_values': 1788,
 'total_values': 4949,
 'duplicated_values': 50639,
 'values_with_whitespace': 47552,
 'values_with_unusual_chars': 416,
 'shortest_value_length': 1.0,
 'longest_value_length': 229.0,
 'average_value_length': 14.572842998585573}

In [61]:
analysis_results_dict[new_col]['shortest_value'][new_col]

1157     1
1463     1
2182     1
2447     1
2906     1
        ..
49160    1
49421    1
50006    1
50402    2
52374    1
Name: edition, Length: 202, dtype: object

In [62]:
analysis_results_dict[new_col]['longest_value'][new_col]

49270    || 36η έκδοση (ανατύπωση): 2002. Νέα, σχολιασμ...
Name: edition, dtype: object

In [63]:
df['edition'] = df['edition'].str.strip().str.lower()
df['edition'] = df['edition'].fillna('unknown')

In [64]:
formats = df['edition'].str.strip().str.lower().value_counts()
formats

edition
unknown                                   47475
first edition                               510
1st edition                                 502
first                                       172
1st                                         158
                                          ...  
beacon paperback 721                          1
penguin freud library                         1
1.0, ebook amazon, b&n, itunes edition        1
cambridge greek and latin classics            1
recopilatorio.                                1
Name: count, Length: 1668, dtype: int64

In [65]:
# Creating a preliminary format_mapping for the 'edition' column
edition_mapping = {
    'first edition': 'first edition',
    '1st edition': 'first edition',
    'first': 'first edition',
    '1st': 'first edition',
    'second edition': 'second edition',
    '2nd edition': 'second edition',
    '2nd': 'second edition',
    'large print': 'large print',
    'penguin classics': 'penguin classics',
    'oxford world\'s classics': 'oxford world classics',
    'unabridged': 'unabridged',
    'revised edition': 'revised edition',
    'omnibus': 'omnibus',
    'abridged': 'abridged',
    'unknown': 'unknown'
}

# Apply the format_mapping to the 'edition' column
df['edition'] = df['edition'].apply(lambda x: edition_mapping.get(x, 'other'))

# Display the unique values for review after the cleaning
df['edition'].value_counts()


edition
unknown                  47475
other                     3009
first edition             1342
second edition             227
large print                103
penguin classics            60
oxford world classics       54
unabridged                  49
revised edition             40
omnibus                     33
abridged                    32
Name: count, dtype: int64

In [66]:
new_col = 'publisher'

analysis_results_dict[new_col]['analysis_results']

{'missing_values': 3692,
 'unique_values': 11110,
 'total_values': 48736,
 'duplicated_values': 41317,
 'values_with_whitespace': 3692,
 'values_with_unusual_chars': 2347,
 'shortest_value_length': 1.0,
 'longest_value_length': 105.0,
 'average_value_length': 15.32669894944189}

In [67]:
analysis_results_dict[new_col]['shortest_value'][new_col]

37412    Q
42894    s
Name: publisher, dtype: object

In [68]:
analysis_results_dict[new_col]['longest_value'][new_col]

27518    Islamic Studies Scholars\n\n202 books — 146 vo...
Name: publisher, dtype: object

In [69]:
df['publisher'].dropna().value_counts()


publisher
Vintage                                      572
HarperCollins                                562
Penguin Books                                430
Ballantine Books                             399
Bantam                                       369
                                            ... 
Council Oak Books                              1
Chicken House / Scholastic                     1
Applause Theatre & Cinema Book Publishers      1
Ahsyad Publications                            1
Bonneville                                     1
Name: count, Length: 11110, dtype: int64

In [70]:
# Convert the 'publisher' column to lowercase
df['publisher'] = df['publisher'].str.strip().str.lower()
df['publisher'] = df['publisher'].str.split(',').str[0]

# Identify publishers that occur more than 30 times
publisher_counts = df['publisher'].value_counts()

frequent_publishers = publisher_counts[publisher_counts > 100]

frequent_publishers_df = frequent_publishers.reset_index()
frequent_publishers_df.columns = ['Publisher', 'Count']
# frequent_publishers_df = frequent_publishers_df.sort_values(by='Publisher')
frequent_publishers_df = frequent_publishers_df[['Publisher']]

# # Display the frequent publishers for inspection
frequent_publishers_df


,Publisher
0,vintage
1,harpercollins
2,little
3,createspace independent publishing platform
4,penguin books
...,...
62,zondervan
63,doubleday
64,g.p. putnam's sons
65,tokyopop


In [71]:
"""from fuzzywuzzy import process
# Create a list of the 100 most common publishers
top_100_publishers = publisher_counts.head(100).index.tolist()

def fuzzy_match_publisher(publisher):
    
    Match the publisher name with the top 100 publishers using fuzzy matching.
    If there's a match with at least 80% similarity, return the matched name.
    Otherwise, return 'other'.
    
    if pd.isna(publisher):
        return 'other'
    
    # Get the closest match from the top 100 publishers
    match, score = process.extractOne(publisher, top_100_publishers)
    
    if score >= 80:
        return match
    else:
        return 'other'

# Apply the fuzzy matching function to the 'publisher' column
df['publisher_cleaned'] = df['publisher'].apply(fuzzy_match_publisher)

# Display a sample of the original and cleaned publisher values for verification
df[['publisher', 'publisher_cleaned']].sample(20)
"""

"from fuzzywuzzy import process\n# Create a list of the 100 most common publishers\ntop_100_publishers = publisher_counts.head(100).index.tolist()\n\ndef fuzzy_match_publisher(publisher):\n    \n    Match the publisher name with the top 100 publishers using fuzzy matching.\n    If there's a match with at least 80% similarity, return the matched name.\n    Otherwise, return 'other'.\n    \n    if pd.isna(publisher):\n        return 'other'\n    \n    # Get the closest match from the top 100 publishers\n    match, score = process.extractOne(publisher, top_100_publishers)\n    \n    if score >= 80:\n        return match\n    else:\n        return 'other'\n\n# Apply the fuzzy matching function to the 'publisher' column\ndf['publisher_cleaned'] = df['publisher'].apply(fuzzy_match_publisher)\n\n# Display a sample of the original and cleaned publisher values for verification\ndf[['publisher', 'publisher_cleaned']].sample(20)\n"

In [72]:
# Create a distionary with mapping for some similar names
publisher_mapping = {
    "amazon digital services": "amazon",
    "atria books": "atria",
    "ballantine books": "ballantine",
    "berkley": "berkley",
    "berkley books": "berkley",
    "bethany house publishers": "bethany house",
    "createspace": "createspace independent publishing platform",
    "createspace independent publishing platform": "createspace independent publishing platform",
    "harper": "harpercollins",
    "harper perennial": "harpercollins",
    "harpercollins": "harpercollins",
    "harpercollins publishers": "harpercollins",
    "harperteen": "harperteen",
    "hmh books for young readers": "hmh",
    "little, brown and company": "little brown",
    "little, brown books for young readers": "little brown",
    "penguin": "penguin",
    "penguin books": "penguin",
    "penguin classics": "penguin",
    "penguin group (usa)": "penguin",
    "scholastic": "scholastic inc.",
    "scholastic inc.": "scholastic inc.",
    "scholastic paperbacks": "scholastic inc.",
    "scholastic press": "scholastic inc.",
    "simon & schuster": "simon and schuster",
    "simon pulse": "simon and schuster",
    "simon schuster": "simon and schuster",
    "simon schuster books for young readers": "simon and schuster",
    "st. martin's griffin": "st. martin",
    "st. martin's paperbacks": "st. martin",
    "st. martin's press": "st. martin",
    "william morrow": "william morrow",
    "william morrow paperbacks": "william morrow"
}

# Map and standardize the 'publisher' column using the publisher_mapping dictionary
df['publisher'] = df['publisher'].map(publisher_mapping).fillna(df['publisher'])

# Checking the first few rows of the dataframe to verify the changes
df['publisher'].head(20)

0                      scholastic inc.
1                      scholastic inc.
2     harper perennial modern classics
3                       modern library
4                               little
5                      alfred a. knopf
6                      signet classics
7                        harpercollins
8                           ballantine
9                         warner books
10                        dutton books
11                             del rey
12                       harpercollins
13                              norton
14                              anchor
15                   vintage books usa
16        random house: modern library
17                             penguin
18                             penguin
19                     signet classics
Name: publisher, dtype: object

In [73]:
# Display the unique values for review after the refined cleaning
df['publisher'].value_counts()

publisher
harpercollins                                  1140
penguin                                         943
createspace independent publishing platform     817
simon and schuster                              700
scholastic inc.                                 612
                                               ... 
andi publisher                                    1
james ward - jmw studios                          1
bang ediciones                                    1
dakota press                                      1
bonneville                                        1
Name: count, Length: 10590, dtype: int64

In [74]:
df['publisher'] = df['publisher'].str[:30]

In [75]:
# 9999999999999 used instead of NA/missing
df['isbn'].value_counts()

isbn
9999999999999    4350
9780765326355       2
9781250166548       2
9780312429980       2
9780439023481       1
                 ... 
9781563899294       1
9780976140733       1
9780446605151       1
9780316036184       1
9781599554976       1
Name: count, Length: 48072, dtype: int64

In [76]:
df['isbn'].isnull().sum()

0

In [77]:
# ISBN should be only numbers, so ignore any that have anything other than numbers
try:
    df['isbn'] = pd.to_numeric(df['isbn'], errors='coerce')
    df['isbn'] = df['isbn'].fillna(9999999999999)
    df['isbn'] = df['isbn'].astype(int)
except Exception as e:
    error_message = str(e)

df['isbn'].head(), error_message if 'error_message' in locals() else None

(0    9780439023481
 1    9780439358071
 2    9999999999999
 3    9999999999999
 4    9780316015844
 Name: isbn, dtype: int64,
 None)

In [79]:
# Convert string representation of lists into actual lists
# df['awards'] = df['awards'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['awards'] = df['awards'].apply(convert_string_to_list)

# Create a new column 'num_awards' that counts the number of awards each book has won
df['num_awards'] = df['awards'].apply(len)

df[['awards', 'num_awards']].head(10)

,awards,num_awards
0,[Locus Award Nominee for Best Young Adult Book...,41
1,[Bram Stoker Award for Works for Young Readers...,9
2,"[Pulitzer Prize for Fiction (1961), Audie Awar...",4
3,[],0
4,"[Georgia Peach Book Award (2007), Buxtehuder B...",25
5,[National Jewish Book Award for Children’s and...,20
6,"[Prometheus Hall of Fame Award (2011), Retro H...",2
7,[],0
8,[],0
9,"[Pulitzer Prize for Novel (1937), National Boo...",2


In [80]:
# Convert string representation of lists into actual lists
# df['characters'] = df['characters'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['characters'] = df['characters'].apply(convert_string_to_list)
# Create a new column 'num_awards' that counts the number of awards each book has won
df['num_characters'] = df['characters'].apply(len)

df[['characters', 'num_awards']].head(10)

,characters,num_awards
0,"[Katniss Everdeen, Peeta Mellark, Cato (Hunger...",41
1,"[Sirius Black, Draco Malfoy, Ron Weasley, Petu...",9
2,"[Scout Finch, Atticus Finch, Jem Finch, Arthur...",4
3,"[Mr. Bennet, Mrs. Bennet, Jane Bennet, Elizabe...",0
4,"[Edward Cullen, Jacob Black, Laurent, Renee, B...",25
5,"[Liesel Meminger, Hans Hubermann, Rudy Steiner...",20
6,"[Snowball, Napoleon, Clover, Boxer, Old Major,...",2
7,"[Polly, Aslan, Lucy Pevensie, Edmund Pevensie,...",0
8,"[Frodo Baggins, Gandalf, Bilbo Baggins, Gollum]",0
9,"[Scarlett O'Hara, Rhett Butler, Ashley Wilkes,...",2


In [81]:
# Keep only the first 3 characters
df['characters'] = df['characters'].apply(lambda x: x[:3] if isinstance(x, list) else [])

# Display the first few rows for verification
df[['characters', 'num_characters']].head(10)

,characters,num_characters
0,"[Katniss Everdeen, Peeta Mellark, Cato (Hunger...",23
1,"[Sirius Black, Draco Malfoy, Ron Weasley]",44
2,"[Scout Finch, Atticus Finch, Jem Finch]",14
3,"[Mr. Bennet, Mrs. Bennet, Jane Bennet]",24
4,"[Edward Cullen, Jacob Black, Laurent]",17
5,"[Liesel Meminger, Hans Hubermann, Rudy Steiner]",8
6,"[Snowball, Napoleon, Clover]",10
7,"[Polly, Aslan, Lucy Pevensie]",30
8,"[Frodo Baggins, Gandalf, Bilbo Baggins]",4
9,"[Scarlett O'Hara, Rhett Butler, Ashley Wilkes]",29


In [82]:
# Apply the function to the 'main_characters' column
df['characters'] = df['characters'].astype(str).apply(remove_brackets_content)

# Display the first few rows for verification
df['characters'].head(10)

0       ['Katniss Everdeen', 'Peeta Mellark', 'Cato ']
1      ['Sirius Black', 'Draco Malfoy', 'Ron Weasley']
2        ['Scout Finch', 'Atticus Finch', 'Jem Finch']
3         ['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet']
4          ['Edward Cullen', 'Jacob Black', 'Laurent']
5    ['Liesel Meminger', 'Hans Hubermann', 'Rudy St...
6                   ['Snowball', 'Napoleon', 'Clover']
7                  ['Polly', 'Aslan', 'Lucy Pevensie']
8        ['Frodo Baggins', 'Gandalf', 'Bilbo Baggins']
9    ["Scarlett O'Hara", 'Rhett Butler', 'Ashley Wi...
Name: characters, dtype: object

In [84]:
df['characters'] = df['characters'].str.strip()

In [85]:
# Convert string representation of lists into actual lists
# df['setting'] = df['setting'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['setting'] = df['setting'].apply(convert_string_to_list)
# Create a new column 'num_awards' that counts the number of awards each book has won
df['num_setting'] = df['setting'].apply(len)

df[['setting', 'num_setting']].head(10)

,setting,num_setting
0,"[District 12, Panem, Capitol, Panem, Panem (Un...",3
1,[Hogwarts School of Witchcraft and Wizardry (U...,2
2,"[Maycomb, Alabama (United States)]",1
3,"[United Kingdom, Derbyshire, England (United K...",4
4,"[Forks, Washington (United States), Phoenix, A...",3
5,"[Molching (Germany), Germany]",2
6,"[England, United Kingdom]",2
7,"[London, England]",1
8,[Middle-earth],1
9,"[Atlanta, Georgia (United States)]",1


In [86]:
# Standardize the genre names to lowercase and remove potential duplicate genres for each book
df['setting'] = df['setting'].apply(lambda x: list(set([setting.lower().strip() for setting in x])) if isinstance(x, list) else x)

# Keep only the first 5 settings
df['setting'] = df['setting'].apply(lambda x: x[:5] if isinstance(x, list) else [])

# Apply the function to the 'main_characters' column
df['setting'] = df['setting'].astype(str).apply(remove_brackets_content)

# Display the first few rows for verification
df[['setting', 'num_setting']].head(10)

,setting,num_setting
0,"['district 12, panem', 'capitol, panem', 'pane...",3
1,"['london, england', 'hogwarts school of witchc...",2
2,"['maycomb, alabama ']",1
3,"['hertfordshire, england ', 'derbyshire, engla...",4
4,"['washington ', 'phoenix, arizona ', 'forks, ...",3
5,"['germany', 'molching ']",2
6,"['united kingdom', 'england']",2
7,"['london, england']",1
8,['middle-earth'],1
9,"['atlanta, georgia ']",1


In [87]:
# Convert string representation of lists into actual lists
# df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['genres'] = df['genres'].apply(convert_string_to_list)
# 2. Standardize the genre names to lowercase and remove potential duplicate genres for each book
df['genres'] = df['genres'].apply(lambda x: list(set([genre.lower().strip() for genre in x])) if isinstance(x, list) else x)

# 3. Handle missing values by ensuring we have lists (even if they are empty)
# df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])

# Create a new column 'num_awards' that counts the number of awards each book has won
df['num_genres'] = df['genres'].apply(len)

df[['genres', 'num_genres']].head(10)

,genres,num_genres
0,"[young adult, fiction, teen, fantasy, science ...",10
1,"[audiobook, young adult, fiction, adventure, f...",10
2,"[high school, young adult, fiction, novels, hi...",10
3,"[fiction, historical romance, novels, classic ...",10
4,"[vampires, paranormal, young adult, fiction, t...",10
5,"[audiobook, young adult, war, fiction, world w...",10
6,"[politics, fiction, novels, fantasy, science f...",10
7,"[young adult, fiction, fantasy, christian fict...",10
8,"[young adult, fiction, fantasy, high fantasy, ...",10
9,"[war, fiction, historical romance, novels, rom...",10


In [ ]:
"""# 1. Convert string representation of lists back into actual Python lists
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 2. Standardize the genre names to lowercase and remove potential duplicate genres for each book
df['genres'] = df['genres'].apply(lambda x: list(set([genre.lower().strip() for genre in x])) if isinstance(x, list) else x)

# 3. Handle missing values by ensuring we have lists (even if they are empty)
df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])

# 4. Create new columns
df['num_genres'] = df['genres'].apply(len)
df['main_genre'] = df['genres'].apply(lambda x: x[0] if x else None)

df[['genres', 'num_genres', 'main_genre']].head()
"""

In [89]:
# Split the 'ratingsByStars' column into five separate columns
ratings_columns = ['rating_5_star', 'rating_4_star', 'rating_3_star', 'rating_2_star', 'rating_1_star']

# Convert the string representation of lists to actual lists
# df['ratingsByStars'] = df['ratingsByStars'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['ratingsByStars'] = df['ratingsByStars'].apply(convert_string_to_list)
# Split the list into the respective columns
df[ratings_columns] = pd.DataFrame(df['ratingsByStars'].tolist(), index=df.index).fillna(0).astype(int)

# Replace None values with 0 and convert the new columns to integer type
# df[ratings_columns] = df[ratings_columns].fillna(0).astype(int)
df.drop(columns=['ratingsByStars'], inplace=True)

# Display the first few rows of the dataframe to verify the changes
df[ratings_columns].head()


,rating_5_star,rating_4_star,rating_3_star,rating_2_star,rating_1_star
0,3444695,1921313,745221,171994,93557
1,1593642,637516,222366,39573,14526
2,2363896,1333153,573280,149952,80794
3,1617567,816659,373311,113934,76770
4,1751460,1113682,1008686,542017,548674


In [90]:
df[ratings_columns].dtypes

rating_5_star    int64
rating_4_star    int64
rating_3_star    int64
rating_2_star    int64
rating_1_star    int64
dtype: object

In [94]:
df['bbeVotes'].describe() #52478.000000

count    52424.000000
mean        22.551160
std        369.347978
min         -4.000000
25%          1.000000
50%          1.000000
75%          2.000000
max      30516.000000
Name: bbeVotes, dtype: float64

In [95]:
# Replace negative values in 'bbeVotes' column with 0
df.loc[df['bbeVotes'] < 0, 'bbeVotes'] = 0
# df['bbeVotes'] = df['bbeVotes'].apply(lambda x: 0 if x < 0 else x)


In [96]:
df[['bbeScore','rating']][df['bbeScore'] == 0]

,bbeScore,rating
52470,0,4.14
52471,0,4.07
52472,0,4.16
52473,0,4.00
52474,0,4.19
52475,0,3.70
52476,0,3.85
52477,0,4.02


In [97]:
df['price'].describe()

count     38080
unique     3776
top        5.20
freq        625
Name: price, dtype: object

In [104]:
df.dtypes

bookId                        int64
title                        object
series                       object
author                       object
rating                      float64
description                  object
language                     object
isbn                          int64
genres                       object
characters                   object
bookFormat                   object
edition                      object
pages                        object
publisher                    object
awards                       object
numRatings                    int64
likedPercent                float64
setting                      object
coverImg                     object
bbeScore                      int64
bbeVotes                      int64
price                        object
oldestPublishDate    datetime64[ns]
num_awards                    int64
num_characters                int64
num_setting                   int64
num_genres                    int64
rating_5_star               

# Outlier Checks

In [ ]:
# Calculating the IQR for bbeScore
Q1 = df['bbeScore'].quantile(0.25)
Q3 = df['bbeScore'].quantile(0.75)
IQR = Q3 - Q1

# Calculating the bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtering rows where bbeScore is an outlier
outliers = df[(df['bbeScore'] < lower_bound) | (df['bbeScore'] > upper_bound)]

# Displaying some of the rows with outliers
# outliers_head = outliers.head()
outliers_count = outliers.shape[0]
outliers_count

In [ ]:
# Check for outliers using IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]

outliers_count = outliers.shape[0]
outliers_percentage = (outliers_count / df.shape[0]) * 100

outliers_count, outliers_percentage

In [ ]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')


In [ ]:
################################################################################
################################################################################

# Save data

In [98]:
# Column memory usage
df.memory_usage(deep=True) / (1024 * 1024)  # Convert bytes to MB

Index                 2.415627
bookId                0.399963
title                 4.189208
series                2.534087
author                3.767038
rating                0.399963
description          54.191002
language              3.186930
isbn                  0.399963
genres                5.564514
characters            3.473130
bookFormat            3.309017
edition               3.205267
pages                 2.934047
publisher             3.500738
awards                2.880066
numRatings            0.399963
likedPercent          0.399963
setting               3.223145
coverImg              7.504100
bbeScore              0.399963
bbeVotes              0.399963
price                 2.660874
oldestPublishDate     0.399963
num_awards            0.399963
num_characters        0.399963
num_setting           0.399963
num_genres            0.399963
rating_5_star         0.399963
rating_4_star         0.399963
rating_3_star         0.399963
rating_2_star         0.399963
rating_1

In [103]:
# Save the cleaned dataframe to a new CSV file
df['description'] = df['description'].str[:25] # Only for the description to be saved
file_path = "/Volumes/Workspace/Turing_College/1.3_project/cleaned_best_books_ever.csv"
df.to_csv(file_path, index=False)


In [ ]:
################################################################################
################################################################################

In [ ]:
df_titles = df['author'].str.extract(r'\((.*?)\)')
df_titles[0].unique()


In [ ]:
# Split the 'author' column based on criteria and create a new DataFrame
# df['author'] = df['author'].str.replace(r'\(Goodreads Author\)', '', regex=True)
# df['author'] = df['author'].str.replace(r'\(Editor\)', '', regex=True)
# Create a list of values to extract
# Create a list of values to replace
# values_to_replace = ['(Goodreads Author)', '(Editor)', '(editor)', '(co-author)', '(Author)', '(General Editor)', '(Writer, Artist)', '(Creator)', '(Series Creator)', '(Author & Illustrator)', '(Author and illustrator)', '(co-writer)']
values_to_replace = ['(Goodreads Author)', '(Editor)', '(Author)']

# Use the list of values in the regular expression for replacements
pattern = '|'.join(map(re.escape, values_to_replace))
# df['author'] = df['author'].str.replace(f'({pattern})', '', regex=True)

# pattern = r'\s*\(' + '|'.join(map(re.escape, values_to_replace)) + r'\)'

# Use the pattern in the regular expression for replacements
df['author'] = df['author'].str.replace(pattern, '', regex=True)


author_df = df['author'].str.split(r',', expand=True)

# Rename the columns
author_df.columns = [f'author_{i}' for i in range(1, author_df.shape[1] + 1)]

# result_df = pd.concat([df['other_column'], author_df], axis=1)


author_df

In [ ]:
# Replace values containing '(' with None (NaN)
author_df = author_df.applymap(lambda x: None if x is not None and '(' in x else x)

print(author_df)

In [ ]:
# num_cols = [col for col in df.columns if df[col].nunique() > 12 and df[col].dtype in ["float64", "int64"]]
num_cols = [col for col in df.columns if df[col].dtype in ["float", "int"]]
num_df = df[num_cols]  

In [ ]:
df.head(100)
# Extract any special characters and leave only the 1st unbroken number sequence
# Genres split from list, do onehotencoding for them
# Split awards?
# Book content? 
# Listing table? 
# Count # chatacters, do 1 col for each
# Split authors by ','

In [ ]:
print(df.info())

In [ ]:
"""Some data types are incorrect, especially 'object' dtype as object, because it typically means that the column 
contains values of different data types, which needs to be fix.
To simplify column names re.sub() searches for anything inside the brackets () in the column name and replaces it 
with an empty string. Then .strip() is removes any leading or trailing spaces in the column name.
Lastly, column names are converted to lowercase."""
df.columns = [re.sub(r'\(.*\)', '', c).strip().lower() for c in df.columns]
print(df.info())

In [ ]:
"""Mapping for column names and types"""
columns_dict = {
                    'car_name': 'string',
                    'make': 'string',
                    'model': 'string',
                    'make_year': 'int64',
                    'color': 'string',
                    'body_type': 'string',
                    'fuel_type': 'string',
                    'engine_type': 'string',
                    'transmission': 'string',
                    'transmission_type': 'string',
                    'emission': 'string'
                    }

for column, dtype in columns_dict.items():
    try:
        df[column] = df[column].astype(dtype)
    except ValueError:
        print(f'Column {column} dtype change failed!')

In [ ]:
"""Split the column on separator into 2 parts and select 1st one, because the error case has number as starting
and the 2nd error case is a non number thus it needs to be forced to numeric. In this case coerce makes that value NaN
which can be replaced with .fillna()"""
df["mileage"] = df["mileage"].str.split(" ", expand = True)[0]
df["mileage"] = pd.to_numeric(df["mileage"], errors='coerce', downcast="float")
df["mileage"] = df["mileage"].fillna(df["mileage"].mean())

"""Has wrong encoding by using "," instead of "." for numbers."""
df["mileage_run"] = df["mileage_run"].str.replace(",", ".").astype("float64")

"""Data is from Spinny Website, which is an Indian used car sale website. """
df["price"] = df["price"].str.replace("," , "", regex = True).astype("int64")

"""Prices are in Indian rupees, thus adding Currency column."""
df["currency"] = "INR"
df["currency"] = df["currency"].astype("string")

"""no_of_owners is categorical, while it could be numerical."""
df['no_of_owners'] = pd.to_numeric(df['no_of_owners'].str.extract(r'(\d+)')[0], errors='coerce')

"""Explode creates additional rows where the split happened."""
df['fuel_type'] = df['fuel_type'].str.split('+')
df = df.explode('fuel_type').reset_index(drop=True)
df['fuel_type'] = df['fuel_type'].astype('string')

In [ ]:
print(df.info())

# Checking and handling missing values

# Handle Duplicates

In [ ]:
"""Unique values in Categorical attributes."""
# print(df.select_dtypes(include=["object", "category"]).nunique())
for col in df.select_dtypes(include=["object", "category", "string"]):
    print(f"{col}: {df[col].nunique()}")


In [ ]:
"""Unique values in Numerical attributes."""
print(df.select_dtypes(include=["float", "int"]).nunique())

In [ ]:
"""Checking and dropping duplicate rows. """
print("Number of duplicate rows:", df.duplicated().sum())

In [ ]:
df = df.drop_duplicates()

In [ ]:
# df['fuel_type'] = df['fuel_type'].apply(lambda x: pd.Series(x.split('+'))).stack().reset_index(drop=True)
df['fuel_type'] = df['fuel_type'].apply(lambda x: x.split('+'))

In [ ]:
df = df.explode('fuel_type', ignore_index=True)

In [ ]:
df['fuel_type'] = df['fuel_type'].str.split('+')
# df = df.explode('fuel_type', ignore_index=True)
df = df.explode('fuel_type').reset_index(drop=True)

# Graphical investigation

In [ ]:
for column in categorical_columns:
    plt.figure(figsize=(8, 6))  # Set the size of the figure
    df[column].value_counts().plot(kind='bar', color='skyblue')  # Create a bar plot
    plt.title(f"{column} Value Counts", fontsize=14)  # Set the title of the plot
    plt.xlabel(column, fontsize=12)  # Set the x-axis label
    plt.ylabel("Count", fontsize=12)  # Set the y-axis label
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
    plt.show()  # Display the plot


In [ ]:
# Calculate the number of rows and columns required for the grid
num_rows = 3
num_cols = 4
num_plots = len(categorical_columns)

# Create a new figure with the specified grid layout
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 12))

# Flatten the axes to a 1D array for easy iteration
axes = axes.flatten()

# Loop over the categorical columns and plot each one
for i, column in enumerate(categorical_columns):
    df[column].value_counts().plot(kind='bar', color='skyblue', ax=axes[i])
    axes[i].set_title(f"{column} Value Counts", fontsize=14)
    axes[i].set_xlabel(column, fontsize=12)
    axes[i].set_ylabel("Count", fontsize=12)
    axes[i].tick_params(axis='x', rotation=45)#, ha='right')

# Hide any empty subplots
for i in range(num_plots, num_rows * num_cols):
    axes[i].axis('off')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
"""Check missing values using sns.barplot to visualize missing values in each column"""
plt.figure(figsize=(10, 6))
sns.barplot(x=df.columns, y=df.isnull().sum())
plt.xticks(rotation=90)
plt.xlabel('Columns')
plt.ylabel('Missing Values Count')
plt.title('Missing Values in DataFrame')
plt.show()

In [ ]:
"""Outliers investigation"""
"""Visualizing outliers in numerical columns with boxplots"""
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

plt.figure(figsize=(12, 8))
sns.boxplot(data=df[numerical_columns], orient='h', palette='Set2')
plt.title('Boxplots for Numerical Columns Outliers')
plt.show()


In [ ]:
def winsorize(df, column, lower_quantile=0.05, upper_quantile=0.95):
    lower_limit = df[column].quantile(lower_quantile)
    upper_limit = df[column].quantile(upper_quantile)
    df[column] = df[column].clip(lower=lower_limit, upper=upper_limit)
    return df

"""Applying winsorization to remove outliers by converting them into less extreme values.
Clip method limits the values in the 'price' to lower and upper limits. 
Values below the 5th and above 95th percentiles are replaced by the 5th/95th percentile values accordingly. 
This does not remove the values, rather caps them at certain points. """
df = winsorize(df, 'price', lower_quantile=0.05, upper_quantile=0.95)

In [ ]:
"""Check for outliers again after winsorization."""
plt.figure(figsize=(6, 4))
sns.boxplot(data=df['price'], orient='v', palette='Set2')
plt.title('Boxplot for "price" column after winsorization')
plt.show()

# Normalization

## Normalization - 1NF
- Each column should contain atomic/single values
- no multiple values / no duplicate rows

In [ ]:
"""Checking and dropping duplicate rows. """
print("Number of duplicate rows:", df.duplicated().sum())

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['fuel_type'] = df['fuel_type'].str.split('+').explode().reset_index(drop=True) #.explode('fuel_type', ignore_index=True)

In [ ]:
df['fuel_type'] = df['fuel_type'].str.split('+')
df = df.explode('fuel_type', ignore_index=True)

In [ ]:
df['fuel_type'] = df['fuel_type'].str.split('+').explode('fuel_type', ignore_index=True)
